<a href="https://colab.research.google.com/github/karlbehrensg/rn-transferencia/blob/master/Aprendizaje_por_Transferencia_y_puesta_a_punto_de_par%C3%A1metros_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [1]:
!pip install tensorflow-gpu

In [2]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-04-17 01:24:39--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   138MB/s    in 0.5s    

2020-04-17 01:24:39 (138 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.1.0'

### Descomprimir el dataset de Perros vs Gatos

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Configurar las rutas al dataset

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 1s 0us/step


In [13]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelar el modelo base

In [0]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [15]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compilar el modelo

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [25]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/8
16/16 [==============================] - 15s 936ms/step - loss: 0.8265 - accuracy: 0.4810 - val_loss: 0.7534 - val_accuracy: 0.5920
Epoch 2/8
16/16 [==============================] - 8s 478ms/step - loss: 0.7484 - accuracy: 0.5465 - val_loss: 0.6841 - val_accuracy: 0.6290
Epoch 3/8
16/16 [==============================] - 8s 478ms/step - loss: 0.6883 - accuracy: 0.5960 - val_loss: 0.6202 - val_accuracy: 0.6720
Epoch 4/8
16/16 [==============================] - 8s 478ms/step - loss: 0.6357 - accuracy: 0.6420 - val_loss: 0.5696 - val_accuracy: 0.6990
Epoch 5/8
16/16 [==============================] - 8s 477ms/step - loss: 0.5890 - accuracy: 0.6850 - val_loss: 0.5338 - val_accuracy: 0.7230
Epoch 6/8
16/16 [==============================] - 8s 479ms/step - loss: 0.5466 - accuracy: 0.7215 - val_loss: 0.5002 - val_acc

### Evaluar el modelo de aprendizaje por transferencia

In [26]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.7910000085830688


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [0]:
base_model.trainable = True

In [29]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [33]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/5
16/16 [==============================] - 12s 744ms/step - loss: 0.1740 - accuracy: 0.9340 - val_loss: 0.1470 - val_accuracy: 0.9520
Epoch 2/5
16/16 [==============================] - 8s 484ms/step - loss: 0.0176 - accuracy: 0.9990 - val_loss: 0.1589 - val_accuracy: 0.9510
Epoch 3/5
16/16 [==============================] - 8s 487ms/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.1754 - val_accuracy: 0.9540
Epoch 4/5
16/16 [==============================] - 8s 489ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.1919 - val_accuracy: 0.9560
Epoch 5/5
16/16 [==============================] - 8s 495ms/step - loss: 8.6892e-04 - accuracy: 1.0000 - val_loss: 0.2232 - val_accuracy: 0.9530


### Evaluar el modelo re calibrado

In [34]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

  ...
    to  
  ['...']


In [35]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.953000009059906
